In [88]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup

import pandas as pd
from io import StringIO

from pathlib import Path
import time

import re

In [89]:
driver = webdriver.Chrome()
driver.implicitly_wait(5)
wait = WebDriverWait(driver,10)

In [90]:
# Get URL

url = 'https://transparencia.e-publica.net/epublica-portal/#/florianopolis/portal/despesa/avancado/diaria/fornecedorTable?entidade=1018'
driver.get(url)

In [57]:
def change_range(range_):
    ''' Change date range on website '''
    date_range = driver.find_element(By.CSS_SELECTOR, 'input[ng-model="dateRangePicker.date"]')
    ActionChains(driver).scroll_to_element(date_range).perform()
    ActionChains(driver).scroll_by_amount(delta_x=0,delta_y=-10000 ).perform()
    date_range.click()
    date_range.clear()

    new_range = range_

    driver.execute_script(""" 
                        arguments[0].value = arguments[1];
                        arguments[0].dispatchEvent(new Event('input', { bubbles: true}));
                        arguments[0].dispatchEvent(new Event('change', { bubbles: true}));
                        """, date_range, new_range)
    click_out = driver.find_element(By.CSS_SELECTOR, '#Unidadegestora_chosen > ul').click()
    
    consultar = driver.find_element(By.CSS_SELECTOR, '#page-top > div:nth-child(1) > div > portal-shell > section > div > div.row > div > div > div > div.content-header-filtro > div > div.col-lg-5.col-md-12.col-sm-12.col-xs-12 > div > div.row.epublica-search-row.epublica-search-row-group > div.col-xs-12.col-sm-8.epublica-portal-search-button-panel > div > button').click()

In [7]:
def change_results():
    ''' change results to 300 '''
    # 1. Clica na caixa para abrir o dropdown
    driver.find_element(By.CSS_SELECTOR, "div.chosen-container a.chosen-single").click()

    # 2. Clica na opção "300" que apareceu na lista visual
    # (O seletor abaixo é padrão da biblioteca 'Chosen', pode variar levemente)
    driver.find_element(By.XPATH, "//li[contains(@class, 'active-result') and text()='300']").click()

In [8]:
def is_true_pagination():
    next_page = driver.find_element(By.CSS_SELECTOR, 'div.pagination > a.pagination-next').get_attribute('class')
    
    if 'disabled' in next_page:
        print('Sem paginação')
        return False
    else:
        next_page.click()
        time.sleep(2)
        return True  

In [ ]:

def get_df():
    ''' Lê a tabela, seleciona colunas únicas e constrói o link completo '''
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    
    if not table:
        return pd.DataFrame()

    data = []
    # Prefixo para corrigir o link
    base_url = 'https://transparencia.e-publica.net/epublica-portal/'
    
    tbody = table.find('tbody')
    if tbody:
        rows = tbody.find_all('tr')
        for row in rows:
            # Ignora linhas ocultas (ng-hide)
            if 'ng-hide' in row.get('class', []):
                continue
            
            cols = row.find_all('td')
            # Garante que a linha tem colunas suficientes para não dar erro
            if len(cols) < 8: 
                continue

            # Mapeamento manual dos índices baseado na sua imagem
            # 0: Favorecido, 1: Cargo, 2: Empenhado, 3: Liquidado, 4: Retido, 5: Pago
            # 6: Pago (Duplicado - Ignorar), 7: Detalhes (Vazio/Link - Pegar só o Link)
            
            # Pega o link da última coluna (coluna 7) e monta a URL completa
            link_element = cols[7].find('a', href=True)
            link_completo = None
            if link_element:
                # Concatena base + href (que geralmente começa com #)
                link_completo = base_url + link_element['href']

            row_data = {
                'Favorecido': cols[0].get_text(strip=True),
                'Cargo': cols[1].get_text(strip=True),
                'Empenhado': cols[2].get_text(strip=True),
                'Liquidado': cols[3].get_text(strip=True),
                'Retido': cols[4].get_text(strip=True),
                'Pago': cols[5].get_text(strip=True), # Pegamos apenas o primeiro "Pago"
                'Link': link_completo
            }
            
            data.append(row_data)

    return pd.DataFrame(data)

# Teste
df = get_df()
df.head()

In [44]:

def scraping_diaria(ano_inicio, ano_final):
    dfs_todos_anos = [] # Lista apenas para guardar os DataFrames finais de cada ano
    
    # Loop reverso de anos
    for ano in range(ano_final, ano_inicio - 1, -1):
        print(f"Coletando dados de {ano}...")
        
        # 1. Define a data UMA VEZ por ano
        change_range(f'01/01/{ano} - 31/12/{ano}')
        time.sleep(3) # Espera carregar
        
        # Garante que mostra 300 resultados UMA VEZ por ano
        try:
            change_results()
            time.sleep(2)
        except:
            print("Não foi possível alterar para 300 (talvez poucos resultados)")

        dfs_ano_atual = [] # Lista para guardar as páginas DESTE ano
        active = True
        pagina = 1
        
        while active:
            print(f"  - Extraindo página {pagina}...")
            
            # Pega o dataframe da página atual
            df_pagina = get_df()
            
            # Só adiciona se tiver dados
            if not df_pagina.empty:
                dfs_ano_atual.append(df_pagina)
            
            # Verifica se tem próxima página
            if is_true_pagination():
                time.sleep(2) # Espera carregar a próxima tabela
                pagina += 1
            else:
                active = False
        
        # Compila os dados deste ano
        if dfs_ano_atual:
            df_final_ano = pd.concat(dfs_ano_atual, ignore_index=True)
            df_final_ano['Ano'] = str(ano)
            dfs_todos_anos.append(df_final_ano)
    
    # Compila tudo no final
    if dfs_todos_anos:
        return pd.concat(dfs_todos_anos, ignore_index=True)
    else:
        return pd.DataFrame()


In [92]:
df_2025

,Favorecido,Cargo,Empenhado,Liquidado,Retido,Pago,Link,Ano
0,ADAILSON EDSON CALIXTO,,"556,10","556,10","0,00","556,10",https://transparencia.e-publica.net/epublica-p...,2025
1,ADRIANO ANALDINO FLOR,Vereador,"20.277,38","20.277,38","0,00","20.277,38",https://transparencia.e-publica.net/epublica-p...,2025
2,AFRÂNIO TADEU BOPPRÉ,Vereador,"8.056,19","8.056,19","0,00","8.056,19",https://transparencia.e-publica.net/epublica-p...,2025
3,ALEX FELIX BENÍCIO,Assessor,"926,82","926,82","0,00","926,82",https://transparencia.e-publica.net/epublica-p...,2025
4,ALEXANDRE MAGNO DE JESUS,,"1.397,35","1.397,35","0,00","1.397,35",https://transparencia.e-publica.net/epublica-p...,2025
...,...,...,...,...,...,...,...,...
73,Suzana Rodrigues Teixeira,Assessora Parlamewntar,"185,37","185,37","0,00","185,37",https://transparencia.e-publica.net/epublica-p...,2025
74,THIAGO AFONSO BORGES JUNIOR,Agente Público,"2.276,66","2.276,66","0,00","2.276,66",https://transparencia.e-publica.net/epublica-p...,2025
75,THIAGO ANTUNES,Assessor,"556,10","556,10","0,00","556,10",https://transparencia.e-publica.net/epublica-p...,2025
76,Thiago Cunha Mendes,,"698,68","698,68","0,00","698,68",https://transparencia.e-publica.net/epublica-p...,2025


In [ ]:
df_2025 = scraping_diaria(2025, 2025)
df_2024_2014 = scraping_diaria(2014, 2024)

In [ ]:
df_2024_2014.to_excel('./data/Diarias_2024_2014.xlsx')

In [84]:
# 1. Função auxiliar para pegar o texto do Objeto (Nível 3) - Já tinhamos feito
def get_objeto_resumido(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    termo_busca = re.compile(r'Objeto resumido', re.IGNORECASE)
    label = soup.find(string=termo_busca)
    
    if label:
        parent = label.parent
        # Tenta pegar irmão, texto solto ou próximo elemento
        proxima_tag = parent.find_next_sibling()
        if proxima_tag and proxima_tag.get_text(strip=True):
             return proxima_tag.get_text(strip=True)
        proximo_texto = parent.next_sibling
        if proximo_texto and str(proximo_texto).strip():
            return str(proximo_texto).strip()
        proximo_elemento = parent.find_next()
        if proximo_elemento:
            return proximo_elemento.get_text(strip=True)
    return "Não encontrado"

# 2. Função auxiliar para ler a tabela de diárias do Favorecido (Nível 2)
def extrair_tabela_intermediaria(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table')
    
    lista_diarias = []
    base_url = 'https://transparencia.e-publica.net/epublica-portal/'

    if not table: return []

    tbody = table.find('tbody')
    if tbody:
        rows = tbody.find_all('tr')
        for row in rows:
            if 'ng-hide' in row.get('class', []): continue # Ignora ocultos
            
            cols = row.find_all('td')
            if len(cols) < 7: continue # Garante colunas mínimas

            # Baseado na sua imagem:
            # 0: Unidade Gestora, 1: Numero, 2: Data, 3: Empenhado, 4: Liquidado, 5: Retido, 6: Pago, 7: Link
            
            # Pega o link da última coluna (botão azul de detalhes)
            link_element = cols[-1].find('a', href=True)
            link_completo = base_url + link_element['href'] if link_element else None

            if link_completo:
                dados_diaria = {
                    'Unidade_Gestora': cols[0].get_text(strip=True),
                    'Numero_Empenho': cols[1].get_text(strip=True),
                    'Data_Emissao': cols[2].get_text(strip=True),
                    'Valor_Empenhado': cols[3].get_text(strip=True),
                    'Valor_Pago': cols[6].get_text(strip=True),
                    'Link_Detalhe_Final': link_completo
                }
                lista_diarias.append(dados_diaria)
    
    return lista_diarias

In [85]:
# 3. O LOOP PRINCIPAL (Junta tudo)
dados_consolidados = []

# Loop no seu DF principal (Tabela Fato)
# Pode limitar df_2025.head(5) para testar antes de rodar tudo
for index, row in df_2025.iterrows():
    
    favorecido = row['Favorecido']
    cargo = row['Cargo']
    link_mestre = row['Link']
    
    print(f"Processando: {favorecido}...")
    
    try:
        # Acessa Nível 2: Lista de Diárias do Favorecido
        driver.get(link_mestre)
        time.sleep(3) # Tempo para carregar a tabela
        
        # Extrai os dados da tabela (Nível 2) usando a função que criamos
        diarias_do_favorecido = extrair_tabela_intermediaria(driver.page_source)
        
        # Loop para acessar cada detalhe (Nível 3)
        for diaria in diarias_do_favorecido:
            link_detalhe = diaria['Link_Detalhe_Final']
            
            # Acessa Nível 3: O Objeto Resumido
            driver.get(link_detalhe)
            time.sleep(2)
            
            objeto = get_objeto_resumido(driver.page_source)
            
            # Monta o dicionário final combinando Pai (Favorecido) e Filho (Diária)
            registro_final = {
                'Favorecido': favorecido,
                'Cargo': cargo,
                'Unidade_Gestora': diaria['Unidade_Gestora'],
                'Numero_Empenho': diaria['Numero_Empenho'],
                'Data_Emissao': diaria['Data_Emissao'],
                'Valor_Empenhado': diaria['Valor_Empenhado'],
                'Valor_Pago': diaria['Valor_Pago'],
                'Objeto_Resumido': objeto,
                'Link_Fonte': link_detalhe
            }
            
            dados_consolidados.append(registro_final)
            
    except Exception as e:
        print(f"Erro ao processar {favorecido}: {e}")
        continue

# Cria o DataFrame final (Tabela Dimensão Completa)
df_detalhado = pd.DataFrame(dados_consolidados)
print("Processo Concluído!")
df_detalhado.head()

Processando: ADAILSON EDSON CALIXTO...
Processando: ADRIANO ANALDINO FLOR...
Processando: AFRÂNIO TADEU BOPPRÉ...
Processando: ALEX FELIX BENÍCIO...
Processando: ALEXANDRE MAGNO DE JESUS...
Processando: ANA CAROLINA DA CUNHA MOREIRA...
Processando: ANDRÉ CARLOS DA SILVA...
Processando: ANTONIO XAVIER SPENGLER FILHO...
Processando: BRUNO ZILIOTTO...
Processando: Bianca Cristina de Souza Duarte...
Processando: CAMILA GONÇALVES CASTOR...
Processando: CARLA SIMARA L. S. S AYRES...
Processando: CAROLINE  CHAMPOWSKI CORRÊA...
Processando: CLAUDIA REGINA FERREIRA...
Processando: CLAUDINEI MARQUES...
Processando: CRISTIANO ANDRÉ HOPPE...
Processando: DANIEL ROCHA DE CARVALHO...
Processando: DOUGLAS CARDOSO SILVEIRA...
Processando: DOUGLAS LUIZ BOTELHO...
Processando: EDINA BARBOSA...
Processando: EDINON MANOEL DA ROSA...
Processando: EDUARDO LOCH...
Processando: ELEANDRO BOTELEIRO CAMPODONIO...
Processando: ELITON FELIPE DE SOUZA...
Processando: ELIZEU OURIQUES...
Processando: Felipe Silva Fer

,Favorecido,Cargo,Unidade_Gestora,Numero_Empenho,Data_Emissao,Valor_Empenhado,Valor_Pago,Objeto_Resumido,Link_Fonte
0,ADAILSON EDSON CALIXTO,,Câmara Municipal de Florianópolis,499,21/07/2025,"556,10","556,10","1,5 (uma e meia) diárias para representar esta...",https://transparencia.e-publica.net/epublica-p...
1,ADRIANO ANALDINO FLOR,Vereador,Câmara Municipal de Florianópolis,580,25/08/2025,"1.164,46","1.164,46","1,5 (uma e meia) diárias para visita a diversa...",https://transparencia.e-publica.net/epublica-p...
2,ADRIANO ANALDINO FLOR,Vereador,Câmara Municipal de Florianópolis,101,14/02/2025,"2.717,08","2.717,08","3,5 (três e meia) diárias para tratar de assun...",https://transparencia.e-publica.net/epublica-p...
3,ADRIANO ANALDINO FLOR,Vereador,Câmara Municipal de Florianópolis,337,29/05/2025,"16.395,84","16.395,84",08 (oito) diárias para participar do EVENTO AM...,https://transparencia.e-publica.net/epublica-p...
4,AFRÂNIO TADEU BOPPRÉ,Vereador,Câmara Municipal de Florianópolis,317,13/05/2025,"617,88","617,88","1,0(uma) Diária para visitar a Cidade de Joinv...",https://transparencia.e-publica.net/epublica-p...


In [87]:
# Nome do arquivo final
nome_arquivo = './data/Diarias_2025.xlsx'

# Usando o ExcelWriter para gerenciar múltiplas abas
with pd.ExcelWriter(nome_arquivo, engine='openpyxl') as writer:
    # Aba 1: Tabela Resumo (Fato)
    df_2025.to_excel(writer, sheet_name='2025_por_fav', index=False)
    
    # Aba 2: Tabela Detalhada (Dimensão/Detalhes)
    df_detalhado.to_excel(writer, sheet_name='Detalhes_Diarias_2025', index=False)

print(f"Arquivo '{nome_arquivo}' salvo com sucesso!")

Arquivo './data/Diarias_2025.xlsx' salvo com sucesso!
